In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms.functional import normalize, to_pil_image
import cv2

## Chargement et prétraitement des données

In [ ]:
# Configuration des paramètres
batch_size = 32

# Transformations pour CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Agrandir les images à 224x224 pour ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),  # CIFAR-10 stats
])

# Chargement des données CIFAR-10
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

num_classes = 10

Files already downloaded and verified
Files already downloaded and verified
Nombre de classes : 10


## Construction du modèle

In [3]:
# Modèle ResNet pour les caractéristiques
class ProtoPNet(nn.Module):
    def __init__(self, num_prototypes, num_classes):
        super(ProtoPNet, self).__init__()
        # Backbone CNN (ResNet-34 sans la couche fully connected)
        self.backbone = resnet34(pretrained=True)
        self.backbone.fc = nn.Identity()  # Supprimer la dernière couche FC
        self.feature_dim = 512  # Dimension de sortie de ResNet-34

        # Couche prototype
        self.prototype_layer = nn.Parameter(torch.randn(num_prototypes, self.feature_dim))

        # Couche fully connected
        self.fc = nn.Linear(num_prototypes, num_classes)

    def forward(self, x):
        # Extraire les caractéristiques
        features = self.backbone(x)

        # Calcul des similarités (distance L2)
        distances = torch.cdist(features.unsqueeze(1), self.prototype_layer.unsqueeze(0), p=2) # J'ai retiré le double "unsqueeze(1)" parce que ça compilait pas mais c'est dans l'article normalement
        similarities = -distances.squeeze(2)  # Les similarités sont inversées des distances

        # Agrégation et classification
        max_similarities = similarities.max(dim=1).values
        output = self.fc(max_similarities)

        return output, max_similarities

## Entraînement

In [4]:
# Hyperparamètres
num_prototypes = num_classes * 10  # 10 prototypes par classe
model = ProtoPNet(num_prototypes=num_prototypes, num_classes=num_classes).to("cpu")

# Optimiseur et scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Fonction de perte
def loss_function(outputs, targets, max_similarities, lambda_clst=0.8, lambda_sep=0.2):
    classification_loss = F.cross_entropy(outputs, targets)

    # Perte de regroupement et séparation
    clst_loss = -max_similarities[range(targets.size(0)), targets].mean()
    sep_loss = max_similarities.mean()

    return classification_loss + lambda_clst * clst_loss + lambda_sep * sep_loss

/home/fahol/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fahol/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
for epoch in range(10):  # 20 époques
    model.train()
    total_loss = 0.0

    print(f"\nDébut de l'époque {epoch + 1}")
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        if batch_idx >= len(train_loader)//10: # Limite à 156 car 1563 c'est long
            break

        images, labels = images.to("cpu"), labels.to("cpu")

        # Passer les données à travers le modèle
        outputs, max_similarities = model(images)
        
        # Impression des dimensions des sorties
        print(f"Batch {batch_idx + 1}/{len(train_loader)//10}:")
        print(f"  Sorties du modèle (outputs) : {outputs.shape}")
        print(f"  Max Similarities : {max_similarities.shape}")

        # Calcul de la perte
        loss = loss_function(outputs, labels, max_similarities)
        total_loss += loss.item()

        # Impression de la perte pour le batch
        print(f"  Perte pour ce batch : {loss.item():.4f}")

        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Mise à jour du scheduler
    scheduler.step()

    # Impression de la perte moyenne après chaque époque
    print(f"Époque {epoch + 1} terminée, Perte moyenne : {total_loss / (len(train_loader)//10):.4f}")


Début de l'époque 1
Batch 1/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 21.7934
Batch 2/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 22.0176
Batch 3/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 20.2943
Batch 4/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 18.1684
Batch 5/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 20.1055
Batch 6/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 20.4590
Batch 7/156:
  Sorties du modèle (outputs) : torch.Size([32, 10])
  Max Similarities : torch.Size([32, 100])
  Perte pour ce batch : 

KeyboardInterrupt: 

## Visualisation des prototypes

In [ ]:
# Fonction pour normaliser les images d'origine pour la visualisation
def denormalize_image(image_tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).cuda()
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).cuda()
    return image_tensor * std + mean

def find_prototype_matches(model, train_loader, num_prototypes):
    model.eval()
    prototype_matches = {i: {"image": None, "patch": None, "activation": -float('inf')} for i in range(num_prototypes)}

    with torch.no_grad():
        for images, _ in train_loader:
            images = images.cuda()
            features = model.backbone(images)  # Extraire les caractéristiques
            features = features.unsqueeze(-1).unsqueeze(-1)

            # Calculer les activations des prototypes
            distances = torch.cdist(features, model.prototype_layer, p=2)
            similarities = -distances  # Les similarités sont inversées des distances

            # Pour chaque prototype, trouver le patch avec l'activation maximale
            for prototype_idx in range(num_prototypes):
                max_similarity, max_index = similarities[:, prototype_idx].view(-1).max(0)
                if max_similarity.item() > prototype_matches[prototype_idx]["activation"]:
                    prototype_matches[prototype_idx]["image"] = images[max_index].cpu()
                    prototype_matches[prototype_idx]["patch"] = features[max_index].cpu()
                    prototype_matches[prototype_idx]["activation"] = max_similarity.item()

    return prototype_matches

In [ ]:
def visualize_prototypes(prototype_matches, save_dir="prototypes"):
    import os
    os.makedirs(save_dir, exist_ok=True)

    for idx, match in prototype_matches.items():
        if match["image"] is None:
            continue

        # Convertir le tensor en image
        original_image = denormalize_image(match["image"]).clamp(0, 1)
        original_image_pil = to_pil_image(original_image)

        # Sauvegarder l'image originale
        original_image_pil.save(f"{save_dir}/prototype_{idx}_original.png")

        # Visualiser le prototype
        fig, ax = plt.subplots(1, 1, figsize=(4, 4))
        ax.imshow(original_image_pil)
        ax.set_title(f"Prototype {idx}")
        ax.axis("off")
        plt.tight_layout()
        plt.savefig(f"{save_dir}/prototype_{idx}_visualization.png")
        plt.close(fig)

In [ ]:
def generate_heatmap(image, activation_map):
    # Redimensionner la carte d'activation à la taille de l'image
    activation_map_resized = cv2.resize(activation_map, (image.size[0], image.size[1]))

    # Normaliser la carte pour les valeurs de 0 à 1
    activation_map_resized = (activation_map_resized - activation_map_resized.min()) / (activation_map_resized.max() - activation_map_resized.min())

    # Superposer la carte d'activation à l'image originale
    heatmap = cv2.applyColorMap(np.uint8(255 * activation_map_resized), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    superimposed = cv2.addWeighted(np.array(image), 0.6, heatmap, 0.4, 0)

    return superimposed

def visualize_heatmaps(model, prototype_matches, save_dir="heatmaps"):
    os.makedirs(save_dir, exist_ok=True)

    for idx, match in prototype_matches.items():
        if match["image"] is None:
            continue

        # Convertir le tensor en image
        original_image = denormalize_image(match["image"]).clamp(0, 1)
        original_image_pil = to_pil_image(original_image)

        # Obtenir l'activation maximale pour ce prototype
        activation_map = match["patch"].view(-1).numpy()

        # Générer la carte de chaleur
        heatmap = generate_heatmap(original_image_pil, activation_map)

        # Sauvegarder la carte de chaleur
        plt.imsave(f"{save_dir}/prototype_{idx}_heatmap.png", heatmap)

In [ ]:
# Trouver les correspondances avec les prototypes
prototype_matches = find_prototype_matches(model, train_loader, num_prototypes=num_prototypes)

# Visualiser les prototypes originaux
visualize_prototypes(prototype_matches, save_dir="prototypes")

# Générer et sauvegarder les cartes de chaleur
visualize_heatmaps(model, prototype_matches, save_dir="heatmaps")

## Evaluation

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs, _ = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Précision : {100 * correct / total:.2f}%")